In [ ]:
# Обработка данных
import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd
from os import listdir
# Визуализация
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

## Метод сингуляного разложения

In [ ]:
import lab3

## Выбор данных

Источник: https://data.worldbank.org/indicator

Мотивация: сжатие данных позволит выделить основные характеристики стран

Гипотеза: первая главная компонента будет соответствовать развитости страны

In [ ]:
DATA_DIR = 'data/world_bank_data/'

In [ ]:
def read_data(path):
    df = pd.DataFrame()
    for file in sorted(listdir(path)):
        # Читаем файл
        series = pd.read_excel(
            DATA_DIR+file, 
            skiprows=3,
            # Индексом используем код страны,
            # потому что он используется для карты
            index_col='Country Code'
        )
        # Берем среднее за 2017-2019 год, преимущества:
        # 1) Меньше пропущенных значений
        # 2) Небольшое сглаживание 
        series = series.loc[:, '2017':'2019'].mean(axis=1)
        df[file] = series
    df.columns = df.columns.str.replace('.xls', '', regex=False)
    return df

In [ ]:
sorted(listdir(DATA_DIR))

Использованные признаки:

* ВВП на душу по покупательной способности
* Ожидаемая продолжительность жизни
* Процент городского населения
* Общая рождаемость
* Рост населения, %
* Рождаемость среди подростков (15-19)
* Процент занятых в сельском хозяйстве
* Процент занятых в промышленности
* Население младше 14, %
* Население старше 65, %



In [ ]:
df = read_data(DATA_DIR)

In [ ]:
# Так как пропуски неслучайные, их просто удалим
df = df.dropna()

In [ ]:
df.shape

In [ ]:
# Часть зависимостей получалась нелинейной, для них применим логарифм
df['GDP per capita PPP'] = df['GDP per capita PPP'].apply(np.log)
df['Population ages 65+'] = df['Population ages 65+'].apply(np.log)

In [ ]:
# Посмотрим на попарные графики
fig = px.scatter_matrix(df)
fig.update_traces(
    diagonal_visible=False,
    marker_size=2.5
)
fig.update_layout(font_size=1, height=750)
fig.show()

На графиках видны линейные связи почти для всех пар, поэтому можно ожидать большую значимость первой компоненты

In [ ]:
# Нормализация
# Вычесть среднее нужно для разложения
# Деление на std убирает разницу единиц измерения
df_norm = (df - df.mean()) / df.std()

## Применение PCA

In [ ]:
def cov(X):
    return np.dot(X, X.T) / (X.shape[1] - 1)

In [ ]:
X = df_norm.values
cov_matrix = cov(X.T)

In [ ]:
%%time
np.random.seed(0)
eigen_values, eigen_vectors = lab3.get_eigen(cov_matrix)

In [ ]:
# Работает достаточно быстро, хотя и размер матрицы небольшой

**Проценты объясненной дисперсии**

In [ ]:
eigen_values

In [ ]:
explained_variance_ratio = eigen_values / eigen_values.sum()

In [ ]:
# Проценты объясненной дисперсии по компонентам
fig = px.bar(explained_variance_ratio*100)
fig.show()

Первая компонента объясняет 73% общей вариации

In [ ]:
# Накопленные значения объясненной вариации
fig = px.line(explained_variance_ratio.cumsum()*100)
fig.update_traces(mode='lines+markers')
fig.show()

Первые 3 компоненты объясняют почти 90% общей вариации, 5 компонент объясняют 95%

3-5 компонент, скорее всего, будет достаточно

Хотя первая содержит б**о**льшую часть информации, и можно использовать даже её одну

**Матрица трансформаций**

In [ ]:
# Применяем трансформацию
X_transformed = np.dot(X, eigen_vectors.T)
df_transformed = pd.DataFrame(
    X_transformed,
    index=df.index
)
# Переименуем колонки с главными компонентами
df_transformed.columns = 'PC ' + (df_transformed.columns + 1).astype(str)

In [ ]:
original_features = df.columns.values
pc_names = df_transformed.columns.values

In [ ]:
# Объединяем признаки и главные компоненты
df_all = pd.concat([df, df_transformed], axis=1)

## Интерпретация и визуализация

In [ ]:
# График первых 3 главных компонент
fig = px.scatter_3d(
    df_all, 
    x='PC 1',
    y='PC 2',
    z='PC 3',
    color='Life expectancy',
    hover_name=df_transformed.index
)
fig.update_layout(margin_t=0, margin_b=0, 
                  margin_l=0, margin_r=0,
                  height=750)
fig.show()

In [ ]:
# Первая компонента на карте
fig = px.choropleth(df_transformed, 
                    locations=df.index,
                    color='PC 1',
                    hover_name=df.index,
                    projection='miller',
                    scope='world',
                    color_continuous_scale=['blue', 'lightblue', 'white', 'pink', 'red'])

fig.update_layout(margin_t=0, margin_b=0, 
                  margin_l=0, margin_r=0,
                  height=750)
fig.show()

Как и ожидалось, первую компоненту можно интерпретировать как общий уровень развития страны

In [ ]:
# Вторая компонента на карте
fig = px.choropleth(df_transformed, 
                    locations=df.index,
                    color='PC 2',
                    hover_name=df.index,
                    projection='miller',
                    scope='world',
                    color_continuous_scale=['blue', 'lightblue', 'white', 'pink', 'red'][::-1])

fig.update_layout(margin_t=0, margin_b=0, 
                  margin_l=0, margin_r=0,
                  height=750)
fig.show()

Вторую компоненту уже сложнее интерпретировать, не придумал ей название

In [ ]:
# Корреляции признаков и главных компонент
corr = df_all.corr().loc[original_features, pc_names]

In [ ]:
# Точки обозначают признаки, 
# их координаты - корреляции признака 
# с первой и второй компонентами соответственно
fig = px.scatter(
    corr, 
    x='PC 1', 
    y='PC 2', 
    text=corr.index, 
    )
fig.update_traces(textposition='top center')
fig.update_layout(height=750)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

По первой главной компоненте признаки явно делятся на две группы: с положительной и отрицательной корреляцией

К первой группе признаков относятся ВВП на душу, продолжительность жизни и процент людей старше 65, процент живущих в городах и занятых в промышленности

Ко второй относятся процент занятых в сельском хозяйстве, показатели рождаемости и рост населения

In [ ]:
fig = px.scatter(
    corr, 
    x='PC 3', 
    y='PC 4', 
    text=corr.index, 
    )
fig.update_traces(textposition='top center')
fig.update_layout(height=750)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Третья компонента сильно связана с процентом занятых в промышленности

Видимо, это связано с арабскими странами, такими как ОАЭ и Катар

(они также выделяются на графике первых 3 главных компонент)

## Ограничения метода

Недостаток метода главных компонент в целом:

используются только линейные зависимости, в данной задачи получилось просто использовать логарифм

(хотя есть нелинейный вариант с применением ядер)

-----

Недостатки SVD:
* Хуже работает с разреженными матрицами (для них есть отдельные функции в пакетах)

* Не очень быстро работает на больших объемах (метод Якоби работает за $O(n^3)$ за шаг). Scikit-learn использует [power method](https://https://en.wikipedia.org/wiki/Power_iteration)

----
Мой вариант (отдельно искать коэффициенты и решать полином) начинает плохо сходиться, если увеличить число признаков. Можно попробовать @retry, пока не экспериментировал